In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import string, nltk
from nltk import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from IPython.display import display, Markdown
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dosoa\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\dosoa\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dosoa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dosoa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Lettura Dataset

In [2]:
raw_dataset = pd.read_csv('../data/raw_dataset.csv')

# Conversione Testo (text-clening)

### Conversione in Stringa della Colonna 'text_'

Per prima cosa viene convertita in stringa la feature "text_" per ogni entry del Dataset. Questo tipo di operazione è utile ad assicurarsi che i dati nella colonna 'text_' siano trattati come testo. È utile anche perchè i dati potrebbero essere stati importati con tipi misti o non corretti.


In [3]:
raw_dataset['text_'] = raw_dataset['text_'].astype(str)

### Conversione in lowercase della Colonna 'text_'

Adesso, appurato che la colonna "text_" contiene solo stringhe, si va a effettuare un'operazione di lowercasing, dove ogni lettera contenuta nella colonna "text_" viene convertita in minucola. Questo è fondamentale in analisi del testo e machine learning, dove le differenze tra maiuscole e minuscole spesso non sono significative e possono creare rumore.

In [4]:
raw_dataset['text_'] = raw_dataset['text_'].str.lower()

# Pulizia del testo

**(Da controllare!)** Successivamente viene definita una routine di operazioni utile a migliorare la qualità del testo. Vediamo perché ciascuna di queste operazioni potrebbe essere utile in questo contesto:

**Tokenizzazione:** Suddividere il testo in parole o token è essenziale per analizzare la struttura e il contenuto delle recensioni. In analisi del testo, specialmente in contesti come il rilevamento delle recensioni false, è importante esaminare le parole individuali per identificare modelli, uso di linguaggio insolito, o altre caratteristiche che potrebbero indicare una recensione falsa.

**Rimozione delle Stop Words:** Le stop words sono generalmente parole comuni che non aggiungono molto significato al testo. Nel contesto del rilevamento di recensioni false, rimuovere queste parole può aiutare a ridurre il rumore nei dati e consentire al modello di concentrarsi su parole più significative che possono essere più indicative di recensioni autentiche o false.

**Rimozione dei Numeri:** I numeri spesso non contribuiscono al rilevamento delle recensioni false, a meno che non siano specificamente rilevanti per il contesto. Rimuovendoli, si riduce ulteriormente il rumore nei dati, permettendo al modello di concentrarsi sul testo chiave.

**Rimozione della Punteggiatura:** La punteggiatura può introdurre rumore e complicazioni nell'analisi del testo. Rimuovendo i segni di punteggiatura, si semplifica il testo per l'analisi. Tuttavia, in alcuni casi, la punteggiatura (come l'eccessivo uso di punti esclamativi) potrebbe essere un indicatore di recensioni false, quindi questa decisione dovrebbe essere basata sulla specifica natura del dataset e sull'obiettivo dell'analisi.

Queste operazioni di pre-elaborazione aiutano a pulire e standardizzare i dati del testo, migliorando la qualità delle informazioni che alimentano il modello di machine learning. Ciò è particolarmente importante nel rilevamento delle recensioni false, dove la precisione e la chiarezza del testo analizzato possono avere un impatto significativo sui risultati del modello.



In [5]:
def text_cleaning(text):
    return ' '.join([word for word in word_tokenize(text) 
                     if word not in stopwords.words('english') 
                     and not word.isdigit() 
                     and word not in string.punctuation])

### Test routine "text_cleaning"

Adesso stampiamo due volte la colonna "text_" delle prime 3 entry del Dataset. La prima volta stampiamo i valori di partenza, mentre la seconda volta andiamo ad applicare la routine "text_cleaning".

In [6]:
# Stampa le prime 3 stringhe senza pulizia
display(Markdown('**Stringhe senza pulizia testo:**'))
for i in range(3):
    string_dirty = raw_dataset['text_'][i]
    display(Markdown(f'{i+1}. {string_dirty}'))

# Stampa le prime 3 stringhe con pulizia
display(Markdown('**Stringhe con pulizia testo:**'))
for i in range(3):
    string_cleaned = text_cleaning(raw_dataset['text_'][i])
    display(Markdown(f'{i+1}. {string_cleaned}'))

**Stringhe senza pulizia testo:**

1. love this!  well made, sturdy, and very comfortable.  i love it!very pretty

2. love it, a great upgrade from the original.  i've had mine for a couple of years

3. this pillow saved my back. i love the look and feel of this pillow.

**Stringhe con pulizia testo:**

1. love well made sturdy comfortable love pretty

2. love great upgrade original 've mine couple years

3. pillow saved back love look feel pillow

Testata la corretta esecuzione della routine, verrà adesso applicata a tutte le entry (routine con tempi di esecuzione molto lunghi):

In [7]:
raw_dataset['text_'] = raw_dataset['text_'].apply(text_cleaning)

## Stemming & Lemmatazing

Lo stemming è il processo di riduzione delle parole alle loro radici o forme base, eliminando prefissi, suffissi e inflessioni. Questo approccio migliora l'analisi del sentimento consolidando varie forme di una parola in una singola radice, aumentando così l'accuratezza e l'efficienza. Ad esempio, "amare", "amato", e "amando" vengono ridotti a "ama-", aiutando l'algoritmo a riconoscere che tutte esprimono sentimenti positivi. Questo riduce la complessità del testo e facilita l'identificazione corretta del sentimento, senza che varianti simili della stessa parola vengano interpretate come concetti distinti.

In [8]:
stemmer = PorterStemmer()
def stem_words(text):
    return ' '.join([stemmer.stem(word) for word in text.split()])
raw_dataset['text_'] = raw_dataset['text_'].apply(lambda x: stem_words(x))

In [9]:
lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return ' '.join([lemmatizer.lemmatize(word) for word in text.split()])
raw_dataset["text_"] = raw_dataset["text_"].apply(lambda text: lemmatize_words(text))

Nell'addestramento di un fake review detector basato su machine learning e analisi del sentimento, la lemmatizzazione e lo stemming possono essere molto utili:

1. **Riduzione della Variabilità del Testo**: Le recensioni possono variare molto in termini di stile e uso della lingua. La lemmatizzazione e lo stemming riducono questa variabilità convertendo diverse forme di una parola nella loro forma base o radice. Ciò aiuta l'algoritmo a focalizzarsi sul contenuto essenziale delle recensioni piuttosto che su variazioni linguistiche superficiali.

2. **Miglioramento dell'Analisi del Sentimento**: Questi processi semplificano il testo, rendendo più efficace l'analisi del sentimento. Per esempio, parole come "buono", "buoni", "buona" possono essere ridotte alla loro radice ("buon-" nello stemming) o al lemma ("buono" nella lemmatizzazione), facilitando l'identificazione di sentimenti positivi in maniera coerente.

3. **Efficienza nell'Elaborazione**: Sia lo stemming che la lemmatizzazione riducono il numero di termini unici nel dataset, diminuendo così il carico computazionale. Questo può velocizzare l'addestramento del modello e migliorare la sua efficienza operativa.

4. **Distinguere le Recensioni Autentiche dalle False**: Le recensioni false possono avere caratteristiche linguistiche uniche, come l'uso eccessivo di parole positive o una struttura frasale innaturale. Analizzando il testo semplificato attraverso stemming e lemmatizzazione, l'algoritmo può imparare a riconoscere questi pattern e identificare più efficacemente le recensioni false.

In conclusione, lo stemming e la lemmatizzazione aiutano a pulire e standardizzare il testo delle recensioni, fornendo un input più coerente e gestibile per l'algoritmo di machine learning, il che è cruciale per l'efficacia del fake review detector.



Sì, l'analisi del sentimento può essere molto utile in un fake review detector per vari motivi:

1. **Identificazione di Pattern Anomali**: Le recensioni false spesso presentano eccessi emotivi, come un entusiasmo smisurato o, al contrario, critiche eccessivamente negative. L'analisi del sentimento può aiutare a identificare questi estremi come possibili indicatori di falsità.

2. **Coerenza tra Valutazione e Testo**: In alcune recensioni false, può esserci una discrepanza tra il punteggio assegnato (ad esempio, 5 stelle) e il sentimento espresso nel testo. L'analisi del sentimento può rilevare queste incongruenze.

3. **Profilo Emotivo delle Recensioni Autentiche**: Imparando il profilo emotivo tipico delle recensioni autentiche, l'algoritmo può utilizzare l'analisi del sentimento per rilevare deviazioni da questo modello, suggerendo potenziali falsità.

4. **Rilevazione di Manipolazioni Strategiche**: Le recensioni false possono essere create per manipolare la percezione di un prodotto o servizio. L'analisi del sentimento può aiutare a identificare tentativi di manipolazione emotiva, che è una pratica comune nelle recensioni artificiose.

Tuttavia, è importante notare che l'analisi del sentimento da sola non è sufficiente per rilevare recensioni false con precisione. È meglio utilizzarla in combinazione con altre tecniche analitiche e di machine learning per ottenere risultati più accurati.

In [10]:
raw_dataset.to_csv('../data/preprocessed_dataset.csv')